In [3]:
# Import Modules
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Load csv panel data file
file_path_panel = os.path.join("..", "Resources", "DataPanel.csv")

# Read panel csv and store as pandas df
panel_df = pd.read_csv(file_path_panel)

# Preview panel df
panel_df.head()

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
0,Afghanistan,2008,3.724,7.370,0.451,50.80,0.718,0.168,0.882,0.518,0.258
1,Afghanistan,2009,4.402,7.540,0.552,51.20,0.679,0.190,0.850,0.584,0.237
2,Afghanistan,2010,4.758,7.647,0.539,51.60,0.600,0.121,0.707,0.618,0.275
3,Afghanistan,2011,3.832,7.620,0.521,51.92,0.496,0.162,0.731,0.611,0.267
4,Afghanistan,2012,3.783,7.705,0.521,52.24,0.531,0.236,0.776,0.710,0.268


In [5]:
# Load csv mortality data file
file_path_mort = os.path.join("..", "Resources", "MortalityData.csv")

# Read panel csv and store as pandas df
mort_df = pd.read_csv(file_path_mort)

# Preview panel df
mort_df.head()

,Country name,Population 2020,Population 2019,"COVID-19 deaths per 100,000 population in 2020",Median age,Island,Index of exposure to COVID-19 infections in other countries as of March 31,Log of average distance to SARS countries,WHO Western Pacific Region,Female head of government,Index of institutional trust,Gini coefficient of income,"All-cause death count, 2017","All-cause death count, 2018","All-cause death count, 2019","All-cause death count, 2020","Excess deaths in 2020 per 100,000 population, relative to 2017-2019 average"
0,United States,331002647.0,328239523.0,104.451,38.3,0,1.688,9.315,0,0,0.250,47.51,2810927.0,2839076.0,2852747.0,3424996.0,179.22
1,Egypt,102334403.0,100388073.0,7.457,25.3,0,1.627,9.007,0,0,0.446,31.56,NaN,NaN,NaN,NaN,NaN
2,Morocco,36910558.0,36471769.0,20.016,29.6,0,2.336,9.226,0,0,0.397,39.55,NaN,NaN,NaN,NaN,NaN
3,Lebanon,6825442.0,6855713.0,21.508,31.1,0,1.891,8.956,0,0,0.107,31.83,NaN,NaN,NaN,NaN,NaN
4,Saudi Arabia,34813867.0,34268528.0,17.875,31.9,0,1.250,8.897,0,0,0.651,45.90,NaN,NaN,NaN,NaN,NaN


In [11]:
# Create filtered df from panel_df to pull 2019 records (pre-pandemic)
panel_2019_df = panel_df.loc[panel_df.year == 2019, :]

# View filtered df
panel_2019_df

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,Negative affect
11,Afghanistan,2019,2.375,7.697,0.420,52.4,0.394,-0.108,0.924,0.351,0.502
23,Albania,2019,4.995,9.544,0.686,69.0,0.777,-0.099,0.914,0.681,0.274
32,Algeria,2019,4.745,9.337,0.803,66.1,0.385,0.005,0.741,0.585,0.215
50,Argentina,2019,6.086,10.000,0.896,69.0,0.817,-0.211,0.830,0.826,0.319
65,Armenia,2019,5.488,9.522,0.782,67.2,0.844,-0.172,0.583,0.598,0.430
...,...,...,...,...,...,...,...,...,...,...,...
1892,Venezuela,2019,5.081,NaN,0.888,66.7,0.626,NaN,0.839,0.761,0.351
1907,Vietnam,2019,5.467,8.992,0.848,68.1,0.952,-0.126,0.788,0.751,0.186
1919,Yemen,2019,4.197,NaN,0.870,57.5,0.651,NaN,0.798,0.543,0.213
1932,Zambia,2019,3.307,8.155,0.638,55.8,0.811,0.077,0.832,0.743,0.394


In [12]:
# Merge panel df with mortality df
merge_df = pd.merge(panel_2019_df, mort_df, how='left', on='Country name')

merge_df

,Country name,year,Life Ladder,Log GDP per capita,Social support,Healthy life expectancy at birth,Freedom to make life choices,Generosity,Perceptions of corruption,Positive affect,...,Log of average distance to SARS countries,WHO Western Pacific Region,Female head of government,Index of institutional trust,Gini coefficient of income,"All-cause death count, 2017","All-cause death count, 2018","All-cause death count, 2019","All-cause death count, 2020","Excess deaths in 2020 per 100,000 population, relative to 2017-2019 average"
0,Afghanistan,2019,2.375,7.697,0.420,52.4,0.394,-0.108,0.924,0.351,...,8.623,0.0,0.0,0.046,31.000,NaN,NaN,NaN,NaN,NaN
1,Albania,2019,4.995,9.544,0.686,69.0,0.777,-0.099,0.914,0.681,...,9.036,0.0,0.0,0.150,28.960,NaN,NaN,NaN,NaN,NaN
2,Algeria,2019,4.745,9.337,0.803,66.1,0.385,0.005,0.741,0.585,...,9.159,0.0,0.0,0.415,27.620,NaN,NaN,NaN,NaN,NaN
3,Argentina,2019,6.086,10.000,0.896,69.0,0.817,-0.211,0.830,0.826,...,9.717,0.0,0.0,0.091,39.600,NaN,NaN,NaN,NaN,NaN
4,Armenia,2019,5.488,9.522,0.782,67.2,0.844,-0.172,0.583,0.598,...,8.858,0.0,0.0,0.174,33.620,27367.0,25645.0,26252.0,35371.0,302.304
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Venezuela,2019,5.081,NaN,0.888,66.7,0.626,NaN,0.839,0.761,...,9.564,0.0,0.0,0.120,37.800,NaN,NaN,NaN,NaN,NaN
140,Vietnam,2019,5.467,8.992,0.848,68.1,0.952,-0.126,0.788,0.751,...,8.108,1.0,0.0,0.490,40.100,NaN,NaN,NaN,NaN,NaN
141,Yemen,2019,4.197,NaN,0.870,57.5,0.651,NaN,0.798,0.543,...,8.966,0.0,0.0,0.267,36.710,NaN,NaN,NaN,NaN,NaN
142,Zambia,2019,3.307,8.155,0.638,55.8,0.811,0.077,0.832,0.743,...,9.251,0.0,0.0,0.322,57.140,NaN,NaN,NaN,NaN,NaN
